In [ ]:
import numpy as np
import pymc as pm
import pandas as pd
from pytensor import tensor as pt
from matplotlib import pyplot as plt
from sunode.solver import Solver
import sunode
import sunode.wrappers.as_pytensor
import arviz as az

pm.__version__

Test with Lotka - Volterra model.  Copied form 
https://github.com/pymc-devs/sunode

In [ ]:
def lotka_volterra(t, y, p):
    """Right hand side of Lotka-Volterra equation.

    All inputs are dataclasses of sympy variables, or in the case
    of non-scalar variables numpy arrays of sympy variables.
    """
    return {
        'hares': p.alpha * y.hares - p.beta * y.lynx * y.hares,
        'lynx': p.delta * y.hares * y.lynx - p.gamma * y.lynx,
    }


from sunode.symode import SympyProblem
problem = SympyProblem(
    params={
        # We need to specify the shape of each parameter.
        # Any empty tuple corresponds to a scalar value.
        'alpha': (),
        'beta': (),
        'gamma': (),
        'delta': (),
    },
    states={
        # The same for all state variables
        'hares': (),
        'lynx': (),
    },
    rhs_sympy=lotka_volterra,
    derivative_params=[
        # We need to specify with respect to which variables
        # gradients should be computed.
        ('alpha',),
        ('beta',),
    ],
)

# The solver generates uses numba and sympy to generate optimized C functions
# for the right-hand-side and if necessary for the jacobian, adoint and
# quadrature functions for gradients.
from sunode.solver import Solver
solver = Solver(problem, sens_mode=None, solver='BDF')


import numpy as np
tvals = np.linspace(0, 10)
# We can use numpy structured arrays as input, so that we don't need
# to think about how the different variables are stored in the array.
# This does not introduce any runtime overhead during solving.
y0 = np.zeros((), dtype=problem.state_dtype)
y0['hares'] = 1
y0['lynx'] = 0.1

# We can also specify the parameters by name:
solver.set_params_dict({
    'alpha': 0.1,
    'beta': 0.2,
    'gamma': 0.3,
    'delta': 0.4,
})

output = solver.make_output_buffers(tvals)
solver.solve(t0=0, tvals=tvals, y0=y0, y_out=output)

# We can convert the solution to an xarray Dataset
solver.as_xarray(tvals, output).solution_hares.plot()


Ok now try to use PYMC , again following the same documentation

In [ ]:
# read in the data
data = pd.read_csv('ode_data.csv')
data.head()

In [ ]:
# plot the data
plt.plot(data.times, data.lynx, 'o')
plt.plot(data.times, data.hare, 'o')

In [ ]:
# Not really sure if this is correct.
def log_if_pos(x):
    return pt.switch(pt.gt(x,0),pt.log(x),pt.zeros_like(x))

with pm.Model() as model:
    hares_start = pm.TruncatedNormal('hares_start',mu = 10, sigma=50, lower = 0.1)
    lynx_start = pm.TruncatedNormal('lynx_start', mu = 10, sigma=50, lower  = 0.1)
     
    alpha = pm.TruncatedNormal('alpha', mu=0.5, sigma = 1, lower = 0, upper=2)
    beta = pm.TruncatedNormal('beta', mu=0.5, sigma = 1, lower = 0, upper=2)
    gamma = pm.TruncatedNormal('gamma', mu=0.5, sigma = 1, lower = 0, upper=2)
    delta = pm.TruncatedNormal('delta', mu=0.5, sigma = 1, lower = 0, upper=2)
    
    y_hat, _, problem, solver, _, _ = sunode.wrappers.as_pytensor.solve_ivp(
        y0={
        # The initial conditions of the ode. Each variable
        # needs to specify a PyTensor or numpy variable and a shape.
        # This dict can be nested.
            'hares': (hares_start, ()),
            'lynx': (lynx_start, ()),
        },
        params={
        # Each parameter of the ode. sunode will only compute derivatives
        # with respect to PyTensor variables. The shape needs to be specified
        # as well. It it infered automatically for numpy variables.
        # This dict can be nested.
            'alpha': (alpha, ()),
            'beta': (beta, ()),
            'gamma': (gamma, ()),
            'delta': (delta, ()),
            'extra': np.zeros(1),
        },
        # A functions that computes the right-hand-side of the ode using
        # sympy variables.
        rhs=lotka_volterra,
        # The time points where we want to access the solution
        tvals=data.times,
        t0=data.times[0]
    )
    
    # We can access the individual variables of the solution using the
    # variable names.
    pm.Deterministic('hares_mu', y_hat['hares'])
    pm.Deterministic('lynx_mu', y_hat['lynx'])
    
    sd = pm.Exponential('sd',10) 
    # multiplicate error.  This can propduce nans if y goes negative, but it doesnt cause any problems
    # Nevertheless to match the julia version I use 'log_if_pos' .
    pm.LogNormal('hares', mu=log_if_pos(y_hat['hares']), sigma=sd, observed=data.hare)
    pm.LogNormal('lynx', mu=log_if_pos(y_hat['lynx']), sigma=sd, observed=data.lynx)

In [ ]:
with model:
    idata = pm.sample(tune=1000, draws=1000, chains =4,cores = 4)

In [ ]:
az.plot_trace(idata, var_names=['alpha', 'beta', 'gamma', 'delta', 'hares_start', 'lynx_start', 'sd']);

In [ ]:
# list the variables in idata
az.summary(idata, var_names=['alpha', 'beta', 'gamma', 'delta', 'hares_start', 'lynx_start', 'sd'])